In [6]:
import glob
import os
import csv
import pandas as pd
import numpy as np 

def read_speeches():
    all_speech_files = glob.glob('/scratch/group/oit_research_data/stanford_congress/hein-bound/speeches_*.txt')
    CONGRESS_MIN_THRESHOLD = 1
    CONGRESS_MAX_THRESHOLD = 115
    
    speech_files = []
    
    for fn in all_speech_files:
        number = int(fn.rsplit('_', 1)[-1].split('.')[0])
        
        if CONGRESS_MIN_THRESHOLD <= number <= CONGRESS_MAX_THRESHOLD:
            speech_files.append(fn)
            speech_files.sort()
    return speech_files

def read_descriptions():
    all_description_files = glob.glob('/scratch/group/oit_research_data/stanford_congress/hein-bound/descr_*.txt')
    CONGRESS_MIN_THRESHOLD = 1
    CONGRESS_MAX_THRESHOLD = 115
    
    description_files = []
    
    for fn in all_description_files:
        number = int(fn.rsplit('_', 1)[-1].split('.')[0])
        
        if CONGRESS_MIN_THRESHOLD <= number <= CONGRESS_MAX_THRESHOLD:
            description_files.append(fn)
            description_files.sort()
    return description_files
        
def reader(fn):
    print(f'Reading {fn}...')
    return pd.read_csv(fn, sep='|', encoding="ISO-8859-1", error_bad_lines=False, warn_bad_lines=False, quoting=csv.QUOTE_NONE)

def clean_data(all_data):
    all_data = all_data.drop(['chamber', 'speech_id', 'number_within_file', 'speaker', 'first_name'], 1)
    all_data = all_data.drop(['last_name', 'state', 'gender', 'line_start', 'line_end', 'file', 'char_count', 'word_count'], 1)
    all_data['date']=pd.to_datetime(all_data['date'],format='%Y%m%d')
    all_data['year'] = pd.to_datetime(all_data['date']).dt.year
    all_data['5yrperiod'] = np.floor(all_data['year'] / 5) * 5 # round each year to the nearest 5 -- by dividing by 5 and "flooring" to the lowest integer
    all_data = all_data.drop(['date', '5yrperiod'], 1)
    all_data['index'] = np.arange(len(all_data)) # create an 'index' column
    return all_data
    
def import_congressional_data(*args, **kwargs):
    cd = kwargs.get('clean_data', None)
    speech_files = read_speeches()
    description_files = read_descriptions()
    
    speeches_df = pd.concat((reader(fn) for fn in speech_files))
    speeches_df.dropna(how='any', inplace=True)
    
    description_df = pd.concat((reader(fn) for fn in description_files))
    
    all_data = pd.merge(speeches_df, description_df, on = 'speech_id')
    all_data.fillna(0, inplace=True)
    
    if cd == True:
        all_data = clean_data(all_data)
    
    return all_data

congressional_data = import_congressional_data(clean_data = True)

Reading /scratch/group/oit_research_data/stanford_congress/hein-bound/speeches_043.txt...
Reading /scratch/group/oit_research_data/stanford_congress/hein-bound/speeches_044.txt...
Reading /scratch/group/oit_research_data/stanford_congress/hein-bound/speeches_045.txt...
Reading /scratch/group/oit_research_data/stanford_congress/hein-bound/speeches_046.txt...
Reading /scratch/group/oit_research_data/stanford_congress/hein-bound/speeches_047.txt...
Reading /scratch/group/oit_research_data/stanford_congress/hein-bound/speeches_048.txt...
Reading /scratch/group/oit_research_data/stanford_congress/hein-bound/speeches_049.txt...
Reading /scratch/group/oit_research_data/stanford_congress/hein-bound/speeches_050.txt...
Reading /scratch/group/oit_research_data/stanford_congress/hein-bound/speeches_051.txt...
Reading /scratch/group/oit_research_data/stanford_congress/hein-bound/speeches_052.txt...
Reading /scratch/group/oit_research_data/stanford_congress/hein-bound/speeches_053.txt...
Reading /s

In [8]:
import os 
import pandas as pd 
from varname import nameof

def set_dir(data): 
    path = os.getcwd()
    current_folder = os.path.basename(path)
    target_folder = nameof(data) + '_subsets'

    if current_folder != target_folder:
        os.makedirs(target_folder)
        os.chdir(target_folder)
        
def interval_subset(data, col_name, start, end, intv):
    set_dir(data)
    
    start = start
    end = end

    while start <= end:
        start = start + intv
        subset = data[(data[col_name] >= start - intv) & (data[col_name] <= start - 1)]
        
        descr = str(subset[col_name].iloc[0])
        descr_2 = str(subset[col_name]. iloc[-1])
        
        file_name = "stanford_congressional_records_" + descr + "_" + descr_2
        
        subset.to_csv(file_name + ".csv", index = False)
        
interval_subset(congressional_data, 'year', 1870, 2010, 5)

In [ ]:
from multiprocessing import Process, Queue, cpu_count

def parallelize_operation(df, function, n_cores = n):
    split_df = np.array_split(df, n_cores)
    pool = Pool(n)
    df = pd.concat(pool.map(function, split_df))
    pool.close()
    pool.join()
    return df

In [ ]:
https://towardsdatascience.com/tokenize-text-columns-into-sentences-in-pandas-2c08bc1ca790

def sentence_split(df):
    
     pd.concat([Series(row['var2'], row['var1'].split(','))              
                    for _, row in a.iterrows()]).reset_index()
    
    
    s = df['speech'].str.split('.').apply(pd.Series,1).stack()
    s.index = s.index.droplevel(-1) # to line up with df's index
    s.name = 'sentence' # needs a name to join

    del df['speech']
    df = df.join(s)
    del df['index']

    
    return df



In [9]:
import os
import gensim

export_gensim_model('/users/sbuongiorno/triples_in_hansard/ft_14351103', )

def export_congressional_gensim_models(dir_path):
    file_names = []
    
    for fname in os.listdir(dir_path):
        file_names.append(fname)
        
    for fname in file_names:
        imported_data = pd.read_csv(fname)
        
        sentences_df = parallelize_operation(imported_data, #structure_me3)
        
        
        period_model = gensim.models.Word2Vec(sentences = )

    


#for period1 in periodnames:
    
    # just the data in the period in question, then clean it
    #period_data = all_data[all_data['5yrperiod'] == period1]

    sentences_df = parallelize_operation(period_data, structure_me3) #  split speech into sentences
    sentences_df2 = split_sentences3(sentences_df) # split sentences into words
    sentences_df3 = cleanup(sentences_df2) # cleanup punctuation and empty lines
    
    # make a gensim model for that data
    period_model = gensim.models.Word2Vec( 
        sentences = sentences_df3['sentence'],
        workers= n,
        iter = 15,
        min_count = 20, 
        size = 100)  
    
    # save the model with the name of the period
    period_model.save('model-' + str(period1)) 
    
    # load model for each 5 yr period - one period per cycle of the for loop
    #period_model = gensim.models.Word2Vec.load('model-' + str(period1)) # to load a saved model

    # append each period to a larger model of all congress
    if period1 == periodnames[0]:
        congress_model = period_model # for the first time, save period_model as congress model
    else:    
        congress_model.build_vocab(sentences_df3['sentence'], # after the first period, add new period data to the congress model
                               update = True)
        congress_model.train(sentences_df3['sentence'], total_examples=period_model.corpus_count, epochs=period_model.epochs) 

    # store the model with the name of the period
    congress_model.save('congress_model-' + str(startdate) + '-' + str(enddate))

IndentationError: expected an indented block (<ipython-input-9-7fa92082d5b4>, line 23)